In [2]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback


In [3]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [4]:
load_dotenv()
api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"]=api_key

In [58]:
llm=HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct",model_kwargs={"temperature": 0.7, "max_length": 512})

In [59]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

import PyPDF2

In [60]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above test, it is your job to \
Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the test as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [61]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [62]:
prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [63]:
chain=LLMChain(llm=llm,prompt=prompt,output_key="quiz")

In [64]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [65]:
prompt2=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [66]:
chain2=LLMChain(llm=llm,prompt=prompt2,output_key="review")

In [67]:
generate_chain=SequentialChain(chains=[chain,chain2],input_variables=['text','number','subject','tone','response_json'],output_variables=['quiz','review'])

In [86]:
file_path="data.txt"

In [87]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [70]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.\n\nFrom a theoretical viewpoint, probably approximately correc

In [71]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [72]:
NUMBER=5
SUBJECT="machine learning"
TONE='simple'

In [73]:
response=generate_chain(
    {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)

c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [74]:
print(response['quiz'])


Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.

From a theoretical viewpoint, probably approximately correct

In [75]:
import re

match = re.search(r"### RESPONSE_JSON\s*(\{.*\})", response['quiz'], re.DOTALL)
#quiz=json.loads(str(match.group(1)))
response_text=str(match.group(1))
response_text
match = re.search(r"###\s*\w+(.*)", response_text, re.DOTALL)
quiz=json.loads(str(match.group(1)))

In [76]:
quiz

{'1': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Alan Turing',
   'b': 'Arthur Samuel',
   'c': 'Donald Hebb',
   'd': 'Warren McCulloch'},
  'correct': 'b'},
 '2': {'mcq': 'What is the primary goal of machine learning?',
  'options': {'a': 'To mimic human thought processes',
   'b': 'To classify data based on models',
   'c': 'To make predictions for future outcomes',
   'd': 'All of the above'},
  'correct': 'd'},
 '3': {'mcq': 'Which of the following is NOT a part of machine learning?',
  'options': {'a': 'Data mining',
   'b': 'Predictive analytics',
   'c': 'Common sense reasoning',
   'd': 'Statistics'},
  'correct': 'c'},
 '4': {'mcq': 'What is the foundation of machine learning?',
  'options': {'a': 'Natural language processing',
   'b': 'Statistics and mathematical optimization',
   'c': 'Computer vision',
   'd': 'Speech recognition'},
  'correct': 'b'},
 '5': {'mcq': 'Which of the following is a subdiscipline in machine learning?',
  'options': {'

In [77]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [78]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a:Alan Turing | b:Arthur Samuel | c:Donald Hebb | d:Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': 'What is the primary goal of machine learning?',
  'Choices': 'a:To mimic human thought processes | b:To classify data based on models | c:To make predictions for future outcomes | d:All of the above',
  'Correct': 'd'},
 {'MCQ': 'Which of the following is NOT a part of machine learning?',
  'Choices': 'a:Data mining | b:Predictive analytics | c:Common sense reasoning | d:Statistics',
  'Correct': 'c'},
 {'MCQ': 'What is the foundation of machine learning?',
  'Choices': 'a:Natural language processing | b:Statistics and mathematical optimization | c:Computer vision | d:Speech recognition',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is a subdiscipline in machine learning?',
  'Choices': 'a:Exploratory data analysis (EDA) | b:Deep learning | c:Teaching strategies | d:Neural networks',
  'Correct': 'd'}]

In [79]:
quiz_df=pd.DataFrame(quiz_table_data)

In [80]:
quiz_df

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a:Alan Turing | b:Arthur Samuel | c:Donald Heb...,b
1,What is the primary goal of machine learning?,a:To mimic human thought processes | b:To clas...,d
2,Which of the following is NOT a part of machin...,a:Data mining | b:Predictive analytics | c:Com...,c
3,What is the foundation of machine learning?,a:Natural language processing | b:Statistics a...,b
4,Which of the following is a subdiscipline in m...,a:Exploratory data analysis (EDA) | b:Deep lea...,d


In [81]:
quiz_df.to_csv("Machine_learning.csv",index=False)

In [82]:
print(response['review'])


You are an expert english grammarian and writer. Given a Multiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural la

In [83]:
t="Check from an expert English Writer of the above quiz:"
match=re.search(re.escape(t) + r"\s*(.*)",response['review'],re.DOTALL)
str(match.group(1))

'Quiz_Analysis: The complexity of the quiz is beginner level. It does not require any prior knowledge beyond basic machine learning concepts and is suitable for students learning the subject for the first time. The questions are well-structured and clearly convey what is being asked, with no ambiguities or leading tones. The difficulty level of the questions is uniform, with no single question standing out as particularly challenging. The tone of the quiz is simple and direct, making it accessible to students at all levels of proficiency in the English language. The quiz also ensures that the questions are not repeated and confirms the test as well, covering key concepts in machine learning.\n\nQuiz_Update: No changes are necessary to the quiz. It is well-suited to the cognitive and analytical abilities of machine learning students and effectively evaluates their understanding of the subject matter.'